# content
- 回归模型网络拓扑
- 模型训练

In [1]:
import torch 
import torch.autograd as autograd
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(1)

In [9]:
# 回归模型网络拓扑
data = [('me gusta comer en la cafeteria'.split(), 'SPANISH'),
        ('Give it to me'.split(), 'ENGLISH'),
        ('No creo que sea. una buena idea'.split(), 'SPANISH'),
        ('No it is not a good idea to get lost at sea'.split(), 'ENGLISH')]

test_data = [('Yo creo que si'.split(), 'SPANSIH'),
             ('it is lost on me'.split(),'ENGLISH')]

# word 词典 
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]  = len(word_to_ix)
            
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

print('word_to_ix:\n',word_to_ix)
print('label_to_ix:\n',label_to_ix)

# 将sentence转化为Bow_vector表达
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] = 1
    return vec.view(1,-1)
        
def make_target(label, label_to_ix):
    
    return torch.LongTensor([label_to_ix[label]])
    
class BoWClassfier(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(BoWClassfier, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        
        # 线性变换
        self.fc = nn.Linear(self.input_size, self.output_size)
    
    # 重写 forward方法定义前向计算
    def forward(self, inputs):
        
        outs = F.log_softmax(self.fc(inputs), dim=1)
        # dim = 1 按行计算log_softmax
        
        return outs 

word_to_ix:
 {'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea.': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'sea': 23, 'Yo': 24, 'si': 25, 'on': 26}
label_to_ix:
 {'SPANISH': 0, 'ENGLISH': 1}


In [10]:
# 模型训练

# 超参
input_size = len(word_to_ix) # vocab_size
output_size = 2 # num_label

# 实例化模型/定义损失函数/优化器
model = BoWClassfier(input_size, output_size).to(device) 
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    for instance, label in data:
        
        # 开始训练时清理模型参数保存的上一轮训练梯度
        model.zero_grad() 
        
        # 生成bow_vec向量,Variable封装
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix)).to(device)
        
        # 生成target 目标Variable: 0 或 1，Variable封装
        target = autograd.Variable(make_target(label, label_to_ix)).to(device)
        
        # 前向计算
        log_probs = model(bow_vec)
        
        # 计算损失，梯度，优化参数
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

# 测试
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec.to(device))
    print(log_probs)

print(next(model.parameters())[:,word_to_ix['creo']])    

tensor([[-0.1579, -1.9236]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-3.0547, -0.0483]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([ 0.2882, -0.4156], device='cuda:0', grad_fn=<SelectBackward>)
